<a href="https://colab.research.google.com/github/ykitaguchi77/AutoGen/blob/main/Autogen%E3%82%B5%E3%83%B3%E3%83%97%E3%83%AB_%E3%82%B0%E3%83%AB%E3%83%BC%E3%83%97%E3%83%81%E3%83%A3%E3%83%83%E3%83%88%EF%BC%88%E3%83%A1%E3%83%8B%E3%83%A5%E3%83%BC%E8%80%83%E6%A1%88%EF%BC%89.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Autogenについて
Microsoft Researchから登場したマルチエージェントによるLLMアプリケーションを実現するフレームワーク。役割を設定したエージェントを定義することで、エージェントがお互いに相談しながらタスクを実行できる。インターネットアクセスやコードの実行、ファンクションなども活用しながら高度な操作ができる。

- 公式サイト  
  https://microsoft.github.io/autogen/  
- Github  
  https://github.com/microsoft/autogen  
- ブログ  
  AutoGen: Enabling next-generation large language model applications  
  https://www.microsoft.com/en-us/research/blog/autogen-enabling-next-generation-large-language-model-applications/  
- arxivペーパー  
  AutoGen: Enabling Next-Gen LLM Applications via Multi-Agent Conversation  
  https://arxiv.org/abs/2308.08155  

## 注意
- 実行には、OpenAIのAPIキーが必要。
- 複雑な処理や高度な連携にはGPT-4のアクセスが必要。
- コードの修正などが繰り返されるケースではAPIアクセスが多発しコストが嵩むので注意。
- タスクによっては、エージェント間でコードやソースなどを共有する必要があり、大量の入力が発生するため、16Kや32Kに対応したモデルでないと動作しない（Rate Limitエラーにひっかかる）。
- Web情報の取得やデータ分析などは16K、32Kモデル必須。


### STEP 1：準備
pipでインストール。  
ランタイム環境にインストールされてない場合は以下実行。

In [ ]:
%%capture --no-stderr
%pip install pyautogen~=0.1.0

### STEP2：設定
設定はOAI_CONFIGファイルから読み込むこともできるが、今回は直接設定する。  
"api_key"にOpenAIのAPIキーを登録しておく。  
Azureを利用する場合は本家のサンプルを参照のこと。  

In [ ]:
import autogen

config_list = [
    {
        "model": "gpt-4",
        "api_key": "OpenAI_APIKEYに置き換え",
        "api_type": "open_ai",
        "api_base": "https://api.openai.com/v1",
        "api_version": None,
#        "request_timeout": 120,
#        "max_retry_period": 90,
#        "retry_wait_time": 5,
    },
   {
        "model": "gpt-3.5",
        "api_key": "OpenAI_APIKEYに置き換え",
        "api_type": "open_ai",
        "api_base": "https://api.openai.com/v1",
        "api_version": None,
    },
    {
        "model": "gpt-3.5-turbo-16k-0613",
        "api_key": "OpenAI_APIKEYに置き換え",
        "api_type": "open_ai",
        "api_base": "https://api.openai.com/v1",
        "api_version": None,
    },
]

### エージェントの定義
今回の例では、以下の４つのエージェントを定義し、1週間分の夕食の献立を相談しながら決めてもらう。  
- floor_manager  
ユーザーとの対話を担当  
- chef  
メニューを考案する  
- doctor  
医学的立場からメニューを修正する  
- kitchen_manager  
メニューに必要な食材などを調達する
  
「system_message=」でエージェントの役割を定義する。  
  
今回はコードの生成と実行はしないが、必要な場合は  
「code_execution_config={"last_n_messages": 2, "work_dir": "groupchat"},」  
を利用する。  
  
「groupchat = autogen.GroupChat(agents=[floor_manager, chef, doctor, kitchen_manager], messages=[], max_round=12)」  
で、参加させるエージェントと実行ラウンド数を決める。  
  
人間の介入を受け付ける場合は、以下を変更
「human_input_mode="TERMINATE"」→「human_input_mode="ALWAYS"」

In [ ]:
llm_config = {"config_list": config_list}
floor_manager = autogen.UserProxyAgent(
   name="floor_manager",
   system_message="人間との対話を通じて、ChefやDoctor、Kitchen_Managerと相談しながら調理や食材に関する課題を解決してください。",
#   code_execution_config={"last_n_messages": 2, "work_dir": "groupchat"},
   human_input_mode="ALWAYS"
)
chef = autogen.AssistantAgent(
    name="chef",
    system_message="世界中の料理を知り尽くした料理人です。健康面はdoctorが検討するので、考慮する必要はありません。味についてのみ検討したレシピを考案し、最高の料理を提供します。考案したメニューをdoctorと相談して、健康面を考慮しながら修正してください。メニューが決定したらkitchen_managerに食材リストの提出を依頼してください。",
    llm_config=llm_config,
)
doctor = autogen.AssistantAgent(
    name="doctor",
    system_message="chefが提案したメニューを医学的な立場で検証し、Chefに修正を依頼してください。",
    llm_config=llm_config,
)
kitchen_manager = autogen.AssistantAgent(
    name="kitchen_manager",
    system_message="料理にかかる費用や必要な食材や調味料を管理します。chefが考案しメニューから必要な食材を検討し、その調達を指示してください。",
    llm_config=llm_config,
)
groupchat = autogen.GroupChat(agents=[floor_manager, chef, doctor, kitchen_manager], messages=[], max_round=12)
manager = autogen.GroupChatManager(groupchat=groupchat, llm_config=llm_config)

### チャット開始
議論してもらうテーマを与えると議論が開始される。  
  
実行時に「chef(to chat_manager):」のように誰宛ての会話なのかが表示される点に注目。

In [ ]:
floor_manager.initiate_chat(
    manager, message=
    "私は高血圧の持病を持っていますが、食に熱心で、常においしいものを食べたいと考えています。このことを考慮して、1週間分の夕食の献立を考案し、それに必要な食材を調達するためのリストを生成してください。"
)
# type exit to terminate the chat

floor_manager (to chat_manager):

私は高血圧の持病を持っていますが、食に熱心で、常においしいものを食べたいと考えています。このことを考慮して、1週間分の夕食の献立を考案し、それに必要な食材を調達するためのリストを生成してください。

--------------------------------------------------------------------------------
chef (to chat_manager):

分かりました。料理人として高血圧に配慮した美味しい献立を提案しましょう。ただし、最終的な判断はdoctorに委ねます。

1週間の献立：

1. 日曜日: トマトとバジルのパスタ、ガーリックトースト
2. 月曜日: 鶏胸肉のグリル、ブロッコリーとチーズのサラダ
3. 火曜日: 鮭と季節の野菜の刺身、ご飯
4. 水曜日: アボカドとトマトのサラダ、全粒粉パン
5. 木曜日: ひじきと豆腐の炒め物、ご飯
6. 金曜日: 焼き魚、小松菜の炒めもの、ご飯
7. 土曜日: 野菜たっぷりのミネストローネ、バゲット

これらの食事は少ない塩分でも美味しく楽しめるメニューを心がけています。 salt以外のspiceを使っても充分に美味しさを引き立てます。

Doctor、あなたの意見を伺いたいです。このメニューは高血圧のある方に適していますか？修正が必要な箇所があれば教えてください。

--------------------------------------------------------------------------------
doctor (to chat_manager):

Chefさん、高血圧の方に対する配慮を考えてのメニュー、ありがとうございます。

私の提案は次の通りです：

1. 日曜日: トマトとバジルのパスタは良いチョイスですが、ガーリックトーストではなく、オリーブオイルを軽く塗った全粒粉パンをおすすめします。
2. 月曜日: 鶏胸肉のグリルとブロッコリーのサラダは非常に良いですが、チーズは塩分が高めなので避けて、ドレッシングはレモン汁とオリーブオイルのシンプルなものにしましょう。
3. 火曜日: 鮭と季節の野菜の刺身は素晴らしいと思いますが、ご飯の代わりに雑穀米を推奨します